In [0]:
from pyspark.sql.functions import *

In [0]:
df_silver = spark.read.table("nyc_taxi.nyc_schema.silver")
df_zone_lookup = spark.read.table("nyc_taxi.nyc_schema.zone_lookup")

In [0]:
df_joined = df_silver.join(
    broadcast(df_zone_lookup),
    df_silver.PULocationID == df_zone_lookup.location_id,
    how="left"
)

In [0]:
df_pickup_heatmap = df_joined.groupBy(
    "pickup_date", "pickup_hour", "zone"
).agg(
    count("*").alias("trip_count")
).orderBy("pickup_date", "pickup_hour", "zone")

In [0]:
df_pickup_heatmap.write.mode("overwrite").saveAsTable(
    "nyc_taxi.nyc_schema.pickup_heatmap_by_location"
)

In [0]:
%sql
select * from nyc_taxi.nyc_schema.pickup_heatmap_by_location